In [2]:
#standard packages
import pandas as pd
import numpy as np

#webscraping packages & SQLite
import pymongo
from pymongo import MongoClient
from bs4 import BeautifulSoup, SoupStrainer
import requests
import urllib.request

In [3]:
client = MongoClient()
db = client.case_files
cases = db.cases

In [146]:
# cases.drop()

In [4]:
def into_mongo(json):
    for i in json['results']:
        _dict = {'case_id' : i['id'], 'frontend_url': i['frontend_url'], 'case_name': i['name'], 
                      'decision_date': i['decision_date'],
                      'court_name': i['court']['name'], 
                      'court_id': i['court']['id'], 
                      'judges': i['casebody']['data']['judges'],
                      'attorneys': i['casebody']['data']['attorneys'],
                      'case_text': i['casebody']['data']['opinions']}

        x = cases.insert_one(_dict)
        
#         if len(i['casebody']['data']['opinions']) > 1:
#             myquery = { "dissent_case_text": None }
#             newvalues = { "$set": { "dissent_case_text":  i['casebody']['data']['opinions'][1]['text']} }
#             cases.update_one(myquery, newvalues)  

In [101]:
token = 'fb6404d3abaad47e2bd2256c1f55575bd1d1232e'

In [102]:
#url i left off at 
link = next_url
next_url


'https://api.case.law/v1/cases/?body_format=&cite=&court=&court_id=&cursor=cD0xOTcwLTA0LTA0Jm89MjY3&decision_date_max=&decision_date_min=1970-01-01&docket_number=&full_case=true&jurisdiction=&name_abbreviation=&reporter=&search='

In [103]:
r = requests.get(f'{link}', headers={'Authorization': f'Token {token}'})
r.status_code
temp = r.json()
into_mongo(temp)
next_url = temp['next']

In [104]:
while next_url != None:
    r = requests.get(f'{next_url}', headers={'Authorization': f'Token {token}'})
    print(r.status_code)
    temp = r.json()
    into_mongo(temp)
    next_url = temp['next']
    print(next_url)

200
https://api.case.law/v1/cases/?body_format=&cite=&court=&court_id=&cursor=cD0xOTcwLTA0LTA2Jm89MTQ0&decision_date_max=&decision_date_min=1970-01-01&docket_number=&full_case=true&jurisdiction=&name_abbreviation=&reporter=&search=
200
https://api.case.law/v1/cases/?body_format=&cite=&court=&court_id=&cursor=cD0xOTcwLTA0LTA3Jm89NTY%3D&decision_date_max=&decision_date_min=1970-01-01&docket_number=&full_case=true&jurisdiction=&name_abbreviation=&reporter=&search=
200
https://api.case.law/v1/cases/?body_format=&cite=&court=&court_id=&cursor=cD0xOTcwLTA0LTA3Jm89MTU2&decision_date_max=&decision_date_min=1970-01-01&docket_number=&full_case=true&jurisdiction=&name_abbreviation=&reporter=&search=
200
https://api.case.law/v1/cases/?body_format=&cite=&court=&court_id=&cursor=cD0xOTcwLTA0LTA4Jm89NTI%3D&decision_date_max=&decision_date_min=1970-01-01&docket_number=&full_case=true&jurisdiction=&name_abbreviation=&reporter=&search=
200


TypeError: 'NoneType' object is not subscriptable

In [105]:
#pausing at ___ cases... 
for i,x in enumerate(db.cases.find({}, {'decision_date':1})):
    print(i)
    print(x)

0
{'_id': ObjectId('5d66e470e59ddf33befdfe06'), 'decision_date': '1970'}
1
{'_id': ObjectId('5d66e470e59ddf33befdfe07'), 'decision_date': '1970'}
2
{'_id': ObjectId('5d66e470e59ddf33befdfe08'), 'decision_date': '1970'}
3
{'_id': ObjectId('5d66e470e59ddf33befdfe09'), 'decision_date': '1970'}
4
{'_id': ObjectId('5d66e470e59ddf33befdfe0a'), 'decision_date': '1970'}
5
{'_id': ObjectId('5d66e470e59ddf33befdfe0b'), 'decision_date': '1970'}
6
{'_id': ObjectId('5d66e470e59ddf33befdfe0c'), 'decision_date': '1970'}
7
{'_id': ObjectId('5d66e470e59ddf33befdfe0d'), 'decision_date': '1970'}
8
{'_id': ObjectId('5d66e470e59ddf33befdfe0e'), 'decision_date': '1970'}
9
{'_id': ObjectId('5d66e470e59ddf33befdfe0f'), 'decision_date': '1970'}
10
{'_id': ObjectId('5d66e470e59ddf33befdfe10'), 'decision_date': '1970'}
11
{'_id': ObjectId('5d66e470e59ddf33befdfe11'), 'decision_date': '1970'}
12
{'_id': ObjectId('5d66e470e59ddf33befdfe12'), 'decision_date': '1970'}
13
{'_id': ObjectId('5d66e470e59ddf33befdfe13'),

792
{'_id': ObjectId('5d6931dbe59ddf33befe011e'), 'decision_date': '1970-01-09'}
793
{'_id': ObjectId('5d6931dbe59ddf33befe011f'), 'decision_date': '1970-01-09'}
794
{'_id': ObjectId('5d6931dbe59ddf33befe0120'), 'decision_date': '1970-01-09'}
795
{'_id': ObjectId('5d6931dbe59ddf33befe0121'), 'decision_date': '1970-01-09'}
796
{'_id': ObjectId('5d6931dbe59ddf33befe0122'), 'decision_date': '1970-01-09'}
797
{'_id': ObjectId('5d6931dbe59ddf33befe0123'), 'decision_date': '1970-01-09'}
798
{'_id': ObjectId('5d6931dbe59ddf33befe0124'), 'decision_date': '1970-01-09'}
799
{'_id': ObjectId('5d6931dbe59ddf33befe0125'), 'decision_date': '1970-01-09'}
800
{'_id': ObjectId('5d6931dbe59ddf33befe0126'), 'decision_date': '1970-01-09'}
801
{'_id': ObjectId('5d6931dbe59ddf33befe0127'), 'decision_date': '1970-01-09'}
802
{'_id': ObjectId('5d6931dbe59ddf33befe0128'), 'decision_date': '1970-01-09'}
803
{'_id': ObjectId('5d6931dbe59ddf33befe0129'), 'decision_date': '1970-01-09'}
804
{'_id': ObjectId('5d6931

{'_id': ObjectId('5d694580e59ddf33befe0495'), 'decision_date': '1970-01-14'}
1680
{'_id': ObjectId('5d694580e59ddf33befe0496'), 'decision_date': '1970-01-14'}
1681
{'_id': ObjectId('5d694580e59ddf33befe0497'), 'decision_date': '1970-01-14'}
1682
{'_id': ObjectId('5d694580e59ddf33befe0498'), 'decision_date': '1970-01-14'}
1683
{'_id': ObjectId('5d694580e59ddf33befe0499'), 'decision_date': '1970-01-14'}
1684
{'_id': ObjectId('5d694580e59ddf33befe049a'), 'decision_date': '1970-01-14'}
1685
{'_id': ObjectId('5d694580e59ddf33befe049b'), 'decision_date': '1970-01-14'}
1686
{'_id': ObjectId('5d694580e59ddf33befe049c'), 'decision_date': '1970-01-14'}
1687
{'_id': ObjectId('5d694580e59ddf33befe049d'), 'decision_date': '1970-01-14'}
1688
{'_id': ObjectId('5d694580e59ddf33befe049e'), 'decision_date': '1970-01-14'}
1689
{'_id': ObjectId('5d694580e59ddf33befe049f'), 'decision_date': '1970-01-14'}
1690
{'_id': ObjectId('5d694580e59ddf33befe04a0'), 'decision_date': '1970-01-14'}
1691
{'_id': ObjectId

{'_id': ObjectId('5d6e63e0e59ddf33befe0939'), 'decision_date': '1970-01-22'}
2868
{'_id': ObjectId('5d6e63e0e59ddf33befe093a'), 'decision_date': '1970-01-22'}
2869
{'_id': ObjectId('5d6e63e0e59ddf33befe093b'), 'decision_date': '1970-01-22'}
2870
{'_id': ObjectId('5d6e63e0e59ddf33befe093c'), 'decision_date': '1970-01-22'}
2871
{'_id': ObjectId('5d6e63e0e59ddf33befe093d'), 'decision_date': '1970-01-22'}
2872
{'_id': ObjectId('5d6e63e0e59ddf33befe093e'), 'decision_date': '1970-01-22'}
2873
{'_id': ObjectId('5d6e63e0e59ddf33befe093f'), 'decision_date': '1970-01-22'}
2874
{'_id': ObjectId('5d6e63e0e59ddf33befe0940'), 'decision_date': '1970-01-22'}
2875
{'_id': ObjectId('5d6e63e0e59ddf33befe0941'), 'decision_date': '1970-01-22'}
2876
{'_id': ObjectId('5d6e63e0e59ddf33befe0942'), 'decision_date': '1970-01-22'}
2877
{'_id': ObjectId('5d6e63e0e59ddf33befe0943'), 'decision_date': '1970-01-22'}
2878
{'_id': ObjectId('5d6e63e0e59ddf33befe0944'), 'decision_date': '1970-01-22'}
2879
{'_id': ObjectId

4179
{'_id': ObjectId('5d7651648e34f6743246a138'), 'decision_date': '1970-02'}
4180
{'_id': ObjectId('5d7651648e34f6743246a139'), 'decision_date': '1970-02'}
4181
{'_id': ObjectId('5d7651648e34f6743246a13a'), 'decision_date': '1970-02-02'}
4182
{'_id': ObjectId('5d7651648e34f6743246a13b'), 'decision_date': '1970-02-02'}
4183
{'_id': ObjectId('5d7651648e34f6743246a13c'), 'decision_date': '1970-02-02'}
4184
{'_id': ObjectId('5d7651648e34f6743246a13d'), 'decision_date': '1970-02-02'}
4185
{'_id': ObjectId('5d7651648e34f6743246a13e'), 'decision_date': '1970-02-02'}
4186
{'_id': ObjectId('5d7651648e34f6743246a13f'), 'decision_date': '1970-02-02'}
4187
{'_id': ObjectId('5d7651648e34f6743246a140'), 'decision_date': '1970-02-02'}
4188
{'_id': ObjectId('5d7651648e34f6743246a141'), 'decision_date': '1970-02-02'}
4189
{'_id': ObjectId('5d7651648e34f6743246a142'), 'decision_date': '1970-02-02'}
4190
{'_id': ObjectId('5d7651648e34f6743246a143'), 'decision_date': '1970-02-02'}
4191
{'_id': ObjectId(

...

9463
{'_id': ObjectId('5d792b618e34f6743246b5dc'), 'decision_date': '1970-03-06'}
9464
{'_id': ObjectId('5d792b618e34f6743246b5dd'), 'decision_date': '1970-03-06'}
9465
{'_id': ObjectId('5d792b618e34f6743246b5de'), 'decision_date': '1970-03-06'}
9466
{'_id': ObjectId('5d792b618e34f6743246b5df'), 'decision_date': '1970-03-06'}
9467
{'_id': ObjectId('5d792b618e34f6743246b5e0'), 'decision_date': '1970-03-06'}
9468
{'_id': ObjectId('5d792b618e34f6743246b5e1'), 'decision_date': '1970-03-06'}
9469
{'_id': ObjectId('5d792b618e34f6743246b5e2'), 'decision_date': '1970-03-06'}
9470
{'_id': ObjectId('5d792b618e34f6743246b5e3'), 'decision_date': '1970-03-06'}
9471
{'_id': ObjectId('5d792b618e34f6743246b5e4'), 'decision_date': '1970-03-06'}
9472
{'_id': ObjectId('5d792b618e34f6743246b5e5'), 'decision_date': '1970-03-06'}
9473
{'_id': ObjectId('5d792b618e34f6743246b5e6'), 'decision_date': '1970-03-06'}
9474
{'_id': ObjectId('5d792b618e34f6743246b5e7'), 'decision_date': '1970-03-06'}
9475
{'_id': Obj

{'_id': ObjectId('5d792bff8e34f6743246ba9a'), 'decision_date': '1970-03-13'}
10678
{'_id': ObjectId('5d792bff8e34f6743246ba9b'), 'decision_date': '1970-03-13'}
10679
{'_id': ObjectId('5d792bff8e34f6743246ba9c'), 'decision_date': '1970-03-13'}
10680
{'_id': ObjectId('5d792bff8e34f6743246ba9d'), 'decision_date': '1970-03-13'}
10681
{'_id': ObjectId('5d792bff8e34f6743246ba9e'), 'decision_date': '1970-03-13'}
10682
{'_id': ObjectId('5d792bff8e34f6743246ba9f'), 'decision_date': '1970-03-13'}
10683
{'_id': ObjectId('5d792bff8e34f6743246baa0'), 'decision_date': '1970-03-13'}
10684
{'_id': ObjectId('5d792bff8e34f6743246baa1'), 'decision_date': '1970-03-13'}
10685
{'_id': ObjectId('5d792bff8e34f6743246baa2'), 'decision_date': '1970-03-13'}
10686
{'_id': ObjectId('5d792bff8e34f6743246baa3'), 'decision_date': '1970-03-13'}
10687
{'_id': ObjectId('5d792bff8e34f6743246baa4'), 'decision_date': '1970-03-13'}
10688
{'_id': ObjectId('5d792bff8e34f6743246baa5'), 'decision_date': '1970-03-13'}
10689
{'_i

11743
{'_id': ObjectId('5d792ca48e34f6743246bec4'), 'decision_date': '1970-03-23'}
11744
{'_id': ObjectId('5d792ca48e34f6743246bec5'), 'decision_date': '1970-03-23'}
11745
{'_id': ObjectId('5d792ca48e34f6743246bec6'), 'decision_date': '1970-03-23'}
11746
{'_id': ObjectId('5d792ca48e34f6743246bec7'), 'decision_date': '1970-03-23'}
11747
{'_id': ObjectId('5d792ca48e34f6743246bec8'), 'decision_date': '1970-03-23'}
11748
{'_id': ObjectId('5d792ca48e34f6743246bec9'), 'decision_date': '1970-03-23'}
11749
{'_id': ObjectId('5d792ca48e34f6743246beca'), 'decision_date': '1970-03-23'}
11750
{'_id': ObjectId('5d792ca48e34f6743246becb'), 'decision_date': '1970-03-23'}
11751
{'_id': ObjectId('5d792ca48e34f6743246becc'), 'decision_date': '1970-03-23'}
11752
{'_id': ObjectId('5d792ca48e34f6743246becd'), 'decision_date': '1970-03-23'}
11753
{'_id': ObjectId('5d792ca48e34f6743246bece'), 'decision_date': '1970-03-23'}
11754
{'_id': ObjectId('5d792ca48e34f6743246becf'), 'decision_date': '1970-03-23'}
1175

12677
{'_id': ObjectId('5d792d198e34f6743246c26a'), 'decision_date': '1970-03-27'}
12678
{'_id': ObjectId('5d792d198e34f6743246c26b'), 'decision_date': '1970-03-27'}
12679
{'_id': ObjectId('5d792d198e34f6743246c26c'), 'decision_date': '1970-03-27'}
12680
{'_id': ObjectId('5d792d198e34f6743246c26d'), 'decision_date': '1970-03-27'}
12681
{'_id': ObjectId('5d792d198e34f6743246c26e'), 'decision_date': '1970-03-27'}
12682
{'_id': ObjectId('5d792d198e34f6743246c26f'), 'decision_date': '1970-03-27'}
12683
{'_id': ObjectId('5d792d198e34f6743246c270'), 'decision_date': '1970-03-27'}
12684
{'_id': ObjectId('5d792d198e34f6743246c271'), 'decision_date': '1970-03-27'}
12685
{'_id': ObjectId('5d792d1e8e34f6743246c272'), 'decision_date': '1970-03-27'}
12686
{'_id': ObjectId('5d792d1e8e34f6743246c273'), 'decision_date': '1970-03-27'}
12687
{'_id': ObjectId('5d792d1e8e34f6743246c274'), 'decision_date': '1970-03-27'}
12688
{'_id': ObjectId('5d792d1e8e34f6743246c275'), 'decision_date': '1970-03-27'}
1268

13927
{'_id': ObjectId('5d792d848e34f6743246c74c'), 'decision_date': '1970-04-06'}
13928
{'_id': ObjectId('5d792d848e34f6743246c74d'), 'decision_date': '1970-04-06'}
13929
{'_id': ObjectId('5d792d848e34f6743246c74e'), 'decision_date': '1970-04-06'}
13930
{'_id': ObjectId('5d792d848e34f6743246c74f'), 'decision_date': '1970-04-06'}
13931
{'_id': ObjectId('5d792d848e34f6743246c750'), 'decision_date': '1970-04-06'}
13932
{'_id': ObjectId('5d792d848e34f6743246c751'), 'decision_date': '1970-04-06'}
13933
{'_id': ObjectId('5d792d848e34f6743246c752'), 'decision_date': '1970-04-06'}
13934
{'_id': ObjectId('5d792d848e34f6743246c753'), 'decision_date': '1970-04-06'}
13935
{'_id': ObjectId('5d792d848e34f6743246c754'), 'decision_date': '1970-04-06'}
13936
{'_id': ObjectId('5d792d848e34f6743246c755'), 'decision_date': '1970-04-06'}
13937
{'_id': ObjectId('5d792d848e34f6743246c756'), 'decision_date': '1970-04-06'}
13938
{'_id': ObjectId('5d792d848e34f6743246c757'), 'decision_date': '1970-04-06'}
1393